###Get access token by OAuth2

In [233]:
import requests
from requests_oauthlib import OAuth2Session
import json
import pandas as pd

client_id = "androidClient"
client_secret = "secret"
username = "username"
password = "password"

authorization_url = "https://app.atimelogger.com/oauth/token?username={0}&password={1}&grant_type=password".format(username, password)
token = requests.post(authorization_url, auth=(client_id, client_secret))
access_token = json.loads(token.text)['access_token']

### Reference Dictionary
Get a dictionary contain types' **ID, Name, Parent's name** for reference.

In [234]:
r_type = requests.get("https://app.atimelogger.com/api/v2/types", 
                 headers = {'Authorization': "bearer " + access_token})
json_type = json.loads(r_type.text)
# for d in json_type['types']:
#     if d['parent']:
#         d['parent'] = d['parent']['guid']
# data = pd.DataFrame(json_type["types"])
# types = pd.DataFrame([d for d in json_type['types'] if not d['group']]).ix[:, [4,5,2,6]]
# group = pd.DataFrame([d for d in json_type['types'] if d['group']]).ix[:, [4,5,2]]
# pd.merge(types, group, left_on = 'parent', right_on = 'guid', how = 'left')

dict_group = {d['guid']:d['name'] for d in json_type['types'] if d['group']}
dict_type = {}
dict_id = {}
for d in json_type['types']:
    if not d['group']:
        parent_name = dict_group[d['parent']['guid']] if d['parent'] != None else None
        dict_type[d['guid']] = (d['name'], parent_name)
        dict_id[d['name']] = (d['guid'], parent_name)

###Helper Function

In [235]:
def getTime(unixSecond):
    """"
    unixSecond: unix timestamp
    returns: string format of date time for unix timestamp
    """
    return str(datetime.fromtimestamp(unixSecond))

def getDelta(duration):
    """"
    duration: duration in second
    returns: string format of duration
    """
    return str(timedelta(seconds = duration)).split('.')[0]

### Today's Report
Get `today`, `yesterday` as parameters and fetch all intervals of yesterday.

In [250]:
from datetime import datetime, date, timedelta
import copy

today = date.today()
day_delta = timedelta(1)
yesterday = today - day_delta
## Following method will generate 4-hours difference with actual time
# today_s = (today - date(1970,1,1)).total_seconds()
# yesterday_s = (yesterday - date(1970,1,1)).total_seconds()
today_s = today.strftime("%s")
yesterday_s = yesterday.strftime("%s")
params = {'from': int(yesterday_s), 'to': int(today_s)}

r_today = requests.get("https://app.atimelogger.com/api/v2/intervals", 
                 params = params, headers = {'Authorization': "bearer " + access_token})
intervals = json.loads(r_today.text)['intervals']

### Test the correctness of time
interval_time = copy.deepcopy(intervals)
for d in interval_time:
    d['from'] = str(datetime.fromtimestamp(d['from']))
    d['to'] = str(datetime.fromtimestamp(d['to']))
    d['type'] = dict_type[d['type']['guid']]
pd.DataFrame(interval_time).ix[:,[2,4,5,1]].head(6)

,from,to,type,comment
0,2015-07-16 22:06:39,2015-07-16 23:40:41,"(House, Wealth)",Comcast inquiry
1,2015-07-16 22:01:52,2015-07-16 22:06:39,"(Rest, Health)",
2,2015-07-16 20:53:16,2015-07-16 22:01:52,"(House, Wealth)",Comcast inquiry
3,2015-07-16 20:37:53,2015-07-16 20:53:16,"(Bill, Wealth)",
4,2015-07-16 20:18:29,2015-07-16 20:37:53,"(Talk Love, Family)",
5,2015-07-16 20:13:26,2015-07-16 20:18:29,"(Transport, None)",


Extract `comment` as task and calculate its time.

In [241]:
from collections import defaultdict

task = defaultdict(dict)
for d in intervals:
    if d['comment']:
        task[d['comment']]['Group'] = dict_type[d['type']['guid']][1]
        if "Time" in task[d['comment']].keys():
            task[d['comment']]['Time'] += (d['to'] - d['from'])
        else:
            task[d['comment']]['Time'] = (d['to'] - d['from'])

task_list = []
for k, v in task.items():
    task_list.append(dict(Task = k, Group = v['Group'], 
                          Time = getDelta(v['Time'])))
pd.DataFrame(task_list)

,Group,Task,Time
0,Learn,Atimelogger API today's task table,0:50:36
1,Wealth,Comcast inquiry,2:42:38
2,Wealth,Select and buy modem router,1:37:49


In [248]:
import numpy as np
sleep_id = dict_id['Sleep'][0]
rest_id = dict_id['Rest'][0]
meal_id = dict_id["Meal"][0]
rest = np.array([])
meal = np.array([])
for d in intervals:
    if d['type']['guid'] == sleep_id:
        sleep = [getTime(d['from']), 
                 getTime(d['to']),
                 getDelta(d['to'] - d['from'])]
    elif d['type']['guid'] == rest_id:
        rest = np.append(rest, d['to'] - d['from'])
    elif d['type']['guid'] == meal_id:
        meal = np.append(meal, d['to'] - d['from'])
print "Sleep: ", sleep
print "Total Rest Time: ", getDelta(rest.sum())
print "Total Mean Time: ", getDelta(meal.sum())

Sleep:  ['2015-07-16 01:02:28', '2015-07-16 09:13:39', '8:11:11']
Total Rest Time:  0:31:54
Total Mean Time:  1:19:47


### Get access token by basic authorization

In [ ]:
from requests.auth import HTTPBasicAuth
import time

# r = requests.get('https://app.atimelogger.com', auth=HTTPBasicAuth(username, password))
today = datetime.date.today()
day_delta = datetime.timedelta(1)
yesterday = today - day_delta
today_s = (today - datetime.date(1970,1,1)).total_seconds()
print today_s
yesterday_s = (yesterday - datetime.date(1970,1,1)).total_seconds()
data = {'from': int(yesterday_s), 'to': int(today_s)}
r = requests.get("https://app.atimelogger.com/api/v2/intervals", params = data, auth=HTTPBasicAuth(username, password))
print(json.dumps(json.loads(r.text), indent=2))
# print int(time.time())
# print time.gmtime()